In [1]:
param_type = 'motors/wiper-and-washer/windshield-wiper-motor'
param_len = 15

list_url = ['https://www.cardone.com/' + param_type + '/?limit=96&sort=alphaasc&page=' + str(_+1) for _ in range(param_len)]

print('总数量：' + str(len(list_url)))
print()

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_part = html.xpath('//li[@class="productCard productCard--list"]')
                
                # = = = = = = = = = = = = = = =
                
                list_part_number = [_.xpath('./descendant::p[@product-data-sku]/text()')[0].strip() for _ in list_part]

                # = = = = = = = = = = = = = = =
                
                list_url = [_.xpath('./descendant::a[@class="card-figure-link"]/@href')[0] for _ in list_part]

                # = = = = = = = = = = = = = = =
                
                list_application_summary = [part.xpath('./descendant::p[@class="card-text card-text--brand AppSummary"]') for part in list_part]
                
                # = = = = = = = = = = = = = = =
                
                list_application_summary = [''.join(application_summary[0].xpath('./text()')).replace('; ', '\n').strip() if len(application_summary) != 0 else '' for application_summary in list_application_summary]
                
                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame({'status': 'ok',
                                        'Menu': url,
                                        'Part_Number': list_part_number,
                                        'Url': list_url,
                                        'Application_Summary': list_application_summary})
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Menu': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./menu_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：15

https://www.cardone.com/motors/wiper-and-washer/windshield-wiper-motor/?limit=96&sort=alphaasc&page=1  <->  [ok] - 剩余数量：10
https://www.cardone.com/motors/wiper-and-washer/windshield-wiper-motor/?limit=96&sort=alphaasc&page=4  <->  [ok] - 剩余数量：9
https://www.cardone.com/motors/wiper-and-washer/windshield-wiper-motor/?limit=96&sort=alphaasc&page=5  <->  [ok] - 剩余数量：8
https://www.cardone.com/motors/wiper-and-washer/windshield-wiper-motor/?limit=96&sort=alphaasc&page=3  <->  [ok] - 剩余数量：7
https://www.cardone.com/motors/wiper-and-washer/windshield-wiper-motor/?limit=96&sort=alphaasc&page=2  <->  [ok] - 剩余数量：6
https://www.cardone.com/motors/wiper-and-washer/windshield-wiper-motor/?limit=96&sort=alphaasc&page=10  <->  [ok] - 剩余数量：5
https://www.cardone.com/motors/wiper-and-washer/windshield-wiper-motor/?limit=96&sort=alphaasc&page=6  <->  [ok] - 剩余数量：4
https://www.cardone.com/motors/wiper-and-washer/windshield-wiper-motor/?limit=96&sort=alphaasc&page=9  <->  [ok] - 剩余数量：3
https://www.ca